SemCom

# Library

In [2]:
import IPython
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random
import itertools
import os
import torch
#import torchtext; #requires downgrading torch
#pip install torch==2.2.2
#pip install torchtext==0.17.2
import tensorflow as tf
import tensorflow.keras as k
import tf_keras
import sklearn
import sklearn.decomposition
import math
import time

In [3]:
from itertools import combinations
from collections import Counter
from collections import defaultdict
from numpy import trapz
from scipy import spatial, stats
from scipy.integrate import simpson
from scipy.interpolate import splrep, BSpline
from scipy.ndimage import uniform_filter1d
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.metrics import accuracy_score, precision_score, recall_score, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tqdm.autonotebook import tqdm, trange

C:\Users\leona\AppData\Local\Temp\ipykernel_21068\3954474801.py:22: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
import wordninja
from sentence_transformers import SentenceTransformer, util

# S Set of Data Points: MNIST digits

In [5]:
from sklearn.datasets import fetch_openml

In [6]:
def normalize(S):
  scaler = MinMaxScaler(feature_range = (0,1))
  return scaler.fit_transform(S)

In [7]:
mnist = fetch_openml('mnist_784', as_frame=False, cache=False)

In [25]:
S = mnist.data.astype('float32')
S_labels = mnist.target.astype('int64')
df_S_labels = pd.DataFrame(data=S_labels, columns=["S_labels"])

In [26]:
print(len(S))

70000


In [66]:
def print_digit_array(digit):
    for pixels_row in digit.reshape(28,28):
        print(np.array2string(pixels_row).replace('\n',''))

In [67]:
#Second data point is an image of a zero
print_digit_array(S[1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  51. 159. 253. 159.  50.   0.   0.   0.   0.   0.   0.   0.   0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  48. 238. 252. 252. 252. 237.   0.   0.   0.   0.   0.   0.   0.   0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  54. 227. 253. 252. 239. 233. 252.  57.   6.   0.   0.   0.   0.   0.   0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  10.  60. 224. 252. 253. 252. 202.  84. 252. 253. 122.   0.   0.   0.   0.   0.   0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 163. 252. 252. 252. 253. 252. 25

*CIFAR10*

We can continue with a more complex dataset if needed.

E.g., CIFAR10 is an image dataset on 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'

# PCA-based Encoder-Decoder 

In [37]:
def method_pca_variance(X, v):
    '''
    v: explained_variance_threshold
    '''
    # Center the data
    mean = np.mean(X, axis=0)
    X_centered = X - mean

    # Compute SVD
    _, singular_values, all_pca = np.linalg.svd(X_centered, full_matrices=False)

    # Compute the squared singular values for explained variance
    singular_values_squared = singular_values ** 2


    # Compute the explained variance for each principal component
    explained_variance = singular_values_squared / np.sum(singular_values_squared)

    # Compute cumulative explained variance
    cumulative_explained_variance = np.cumsum(explained_variance)

    # Determine the number of components required to reach the threshold
    n_components = np.searchsorted(cumulative_explained_variance, v) + 1

    # Select the top n_components
    pca = all_pca[:n_components, :] #Principal Components (Vt)

    return mean, pca

In [38]:
def pca_encode(X, mean, pca):
    #Encoder
    # Center the new dataset with the same mean vector
    X_centered = X - mean
    # Transform the data using the principal components
    X_encoded = X_centered @ pca.T

    return X_encoded

In [39]:
def pca_decode(X_encoded, mean, pca):
    #Decoder
    # Reconstruct the data
    X_decoded = X_encoded @ pca
    # Reverse centering by adding the mean vector
    X_decoded_centered = X_decoded + mean    
    return X_decoded_centered

In [41]:
def pca_encode_decode_reconstruction_error(X, mean, pca):
    #Encoder
    X_centered = X - mean
    # Transform data (compression)
    X_encoded = X_centered @ pca.T

    #Decoder
    # Reconstruct the data
    X_decoded = X_encoded @ pca #@ = np.dot = matrix multiplication
    # Reverse centering by adding the mean vector
    X_decoded_centered = X_decoded + mean

    #Compute mean squared error between original and decompressed data
    mse = pd.Series(map(mean_squared_error, X, X_decoded_centered))

    return mse

# Proposed Approach

## 1. Train Self-Supervised Encoder-Decoder

In [59]:
mean, pca = method_pca_variance(X=S, v=0.8)

## 2. Encoding (Compression)

In [60]:
S_z = pca_encode(S, mean, pca)

In [ ]:
#compressed second digit
S_z[1]

array([1010.494    , -289.9636   , -576.1207   , -485.08362  ,
        841.47833  , -145.46716  ,  -48.333355 ,  102.231064 ,
        -34.786186 , -151.26501  , -282.70004  , -110.107765 ,
        329.9303   ,  -97.00281  , -226.4773   , -156.05844  ,
        -49.416885 ,  -37.92279  ,   63.192932 ,  113.47844  ,
        202.56096  ,  -94.196815 ,  -12.712128 ,  -29.628128 ,
        233.85992  ,  196.91718  ,   18.939308 ,  -28.379498 ,
        363.63416  ,  -84.49492  ,   -7.011917 , -100.57101  ,
        192.7258   ,   86.99288  , -137.27632  ,  -79.502464 ,
         61.11911  ,    4.0909624,  -43.707153 ,  -83.74587  ,
        219.2553   ,  -64.80328  ,   68.30545  ], dtype=float32)

## 3. Transmission

In [74]:
print("Original size of a digit:")
print(len(S[0]))
print("Compressed size of a digit:")
print(len(S_z[0]))
print("Reduction:")
print(f"{1 - (len(S_z[0])/len(S[0])):.2%}")

print("-------")
print("Original size of a digit times dataset size:")
print(str(len(S[0])) + " x " + str(len(S)) + " = " + str(len(S[0])*len(S)))
print("Compressed size of a digit times dataset size:")
print(str(len(S_z[0])) + " x " + str(len(S)) + " = " + str(len(S_z[0])*len(S)))

Original size of a digit:
784
Compressed size of a digit:
43
Reduction:
94.52%
-------
Original size of a digit times dataset size:
784 x 70000 = 54880000
Compressed size of a digit times dataset size:
43 x 70000 = 3010000


## 4. Decoding (Reconstruction)

In [62]:
S_hat = pca_decode(S_z, mean, pca)

In [72]:
print_digit_array(np.array([round(pixel) for pixel in S_hat[1]]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[  0   0   0   0   0   0   0   0  -1  -1  -2  -4  -7 -10 -10  -6   3  10  13  10   6   2   0   0   0   0   0   0]
[  0   0   0   0   0   0  -1  -2  -4  -7 -10 -16 -22 -26 -23  -7  16  30  31  21  10   3   0   0   0   0   0   0]
[  0   0   0   0   0  -1  -2  -6 -10 -15 -21 -29 -33 -20  23  95 159 179 143  85  32   5  -2  -3  -1   0   0   0]
[  0   0   0   0   0  -1  -3  -6  -9 -14 -23 -33 -29   8  85 182 262 282 232 141  54   2 -13 -11  -4  -1   0   0]
[  0   0   0   0   1   2   4   5   3  -4 -17 -20   9  73 147 212 253 267 242 169  83  18 -10 -12  -4   0   0   0]
[  0   0   0   1   4   9  14  18  13   0  -5  27  99 181 230 239 231 223 213 187 131  69  26   6   4   3   1   0]
[  0   0   1   2   6  11  14  12   2  -5  27 113 215 276 277 245 207 172 164 179 175 133  75  32  12   5   1   0]
[  0   0   0   2   4   4   0 -11 -19   9  96 205 275 270 222 180 144 104  86 131 184 1

In [63]:
data_accuracy = pca_encode_decode_reconstruction_error(S, mean, pca)

In [64]:
data_accuracy

0        1066.612061
1         860.795349
2        1742.022949
3         378.988495
4         943.702026
            ...     
69995     944.032043
69996     611.196838
69997     522.794312
69998    1120.114258
69999    1277.763428
Length: 70000, dtype: float32